<a href="https://colab.research.google.com/github/nissi00/machine-learning/blob/main/appriori42.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
# drive.mount('/content/drive')

# Importation des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import AgglomerativeClustering, KMeans
import warnings

# Ignorer les avertissements
warnings.simplefilter("ignore")
np.random.seed(42)

# Installation et importation de ucimlrepo
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo

# Récupérer le dataset "adult"
adult = fetch_ucirepo(id=2)
X = adult.data.features
y = adult.data.targets

# Renommer la colonne cible pour une meilleure compréhension
y = y.rename(columns={y.columns[0]: "income"})

# Concaténer les données et les cibles en un seul DataFrame
data = pd.concat([X, y], axis=1)
data["income"] = data["income"].replace({">50K.": ">50K", "<=50K.":"<=50K"})


data = data.sample(frac=0.1, random_state=42)
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
7762,18,Private,423024,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,20,United-States,<=50K
23881,17,Private,178953,12th,8,Never-married,Sales,Own-child,White,Female,0,0,20,United-States,<=50K
30507,25,Local-gov,348986,HS-grad,9,Never-married,Handlers-cleaners,Other-relative,Black,Male,0,0,40,United-States,<=50K
28911,20,Private,218215,Some-college,10,Never-married,Sales,Own-child,White,Female,0,0,30,United-States,<=50K
19484,47,Private,244025,HS-grad,9,Never-married,Machine-op-inspct,Unmarried,Amer-Indian-Eskimo,Male,0,0,56,Puerto-Rico,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43114,57,Private,366421,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,Mexico,<=50K
5042,25,Private,371987,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
7027,64,?,143716,Masters,14,Married-civ-spouse,?,Husband,White,Male,0,0,2,United-States,<=50K
36588,23,Private,142566,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K


In [2]:
for column in data.columns:
    if data[column].isnull().sum() > 0:
        if data[column].dtype in ['float64', 'int64']:
            data[column].fillna(data[column].median(), inplace=True)
        else:
            data[column].fillna(data[column].mode()[0], inplace=True)
# To remove the NA values

In [3]:
data.shape

(4884, 15)

In [4]:
records = []
for i in range(4884):
    records.append([ str(data.values[i,j])  for j in range(15)])

In [5]:
records[0]

['18',
 'Private',
 '423024',
 'HS-grad',
 '9',
 'Never-married',
 'Other-service',
 'Not-in-family',
 'White',
 'Male',
 '0',
 '0',
 '20',
 'United-States',
 '<=50K']

In [6]:
!pip install apyori


  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5953 sha256=9cef8ae6e8f77d27c609dd29d134f7d94076758caec1a01780ad84168378f36c
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [7]:
import numpy as np
import pandas as pd
import matplotlib as plt
from apyori import apriori

In [8]:
association_rules = apriori( records,
                           min_supporrt = 0.5,
                           min_confidence = 0.8,
                           min_lift = 2,
                           min_length = 4
                           )

In [9]:
association_results= list(association_rules)


In [10]:
association_results

[RelationRecord(items=frozenset({'10', 'Some-college'}), support=0.22461097461097462, ordered_statistics=[OrderedStatistic(items_base=frozenset({'10'}), items_add=frozenset({'Some-college'}), confidence=0.9699381078691425, lift=4.318302387267905), OrderedStatistic(items_base=frozenset({'Some-college'}), items_add=frozenset({'10'}), confidence=1.0, lift=4.318302387267905)]),
 RelationRecord(items=frozenset({'13', 'Bachelors'}), support=0.16646191646191646, ordered_statistics=[OrderedStatistic(items_base=frozenset({'13'}), items_add=frozenset({'Bachelors'}), confidence=1.0, lift=6.0073800738007375), OrderedStatistic(items_base=frozenset({'Bachelors'}), items_add=frozenset({'13'}), confidence=1.0, lift=6.0073800738007375)]),
 RelationRecord(items=frozenset({'HS-grad', '9'}), support=0.3167485667485668, ordered_statistics=[OrderedStatistic(items_base=frozenset({'9'}), items_add=frozenset({'HS-grad'}), confidence=0.9980645161290324, lift=3.150967741935484), OrderedStatistic(items_base=froze

In [11]:
len(association_results)

428

In [12]:
# convert result in a dataframe for further operation...
df_results = pd.DataFrame(association_results)

df_results

,items,support,ordered_statistics
0,"(10, Some-college)",0.224611,"[((10), (Some-college), 0.9699381078691425, 4...."
1,"(13, Bachelors)",0.166462,"[((13), (Bachelors), 1.0, 6.0073800738007375),..."
2,"(HS-grad, 9)",0.316749,"[((9), (HS-grad), 0.9980645161290324, 3.150967..."
3,"(Married-civ-spouse, Husband)",0.399468,"[((Husband), (Married-civ-spouse), 0.998975934..."
4,"(Never-married, Own-child)",0.141278,"[((Own-child), (Never-married), 0.909090909090..."
...,...,...,...
423,"(Married-civ-spouse, Husband, United-States, M...",0.121007,"[((United-States, <=50K, Private, Husband), (W..."
424,"(Married-civ-spouse, Private, Husband, United-...",0.108722,"[((United-States, Private, Husband, 40), (Whit..."
425,"(HS-grad, United-States, Male, White, <=50K, 0...",0.107699,"[((9, <=50K, Private, Male), (HS-grad, United-..."
426,"(Married-civ-spouse, Husband, HS-grad, United-...",0.111998,"[((9, Husband), (Married-civ-spouse, HS-grad, ..."
